# Retriever Reader Pipeline

We've setup our Elasticsearch instance and are now ready to prepare the remainder of our ODQA pipeline - our *retriever* and *reader*.

First we initialize the connection to our Elasticsearch document store.

In [6]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='squad_docs'
)

Then we initialize our retriever and reader models. We will be using [deepset/bert-base-cased-squad2](https://huggingface.co/deepset/bert-base-cased-squad2) as our reader model.

In [7]:
from haystack.nodes import BM25Retriever
from haystack.nodes import FARMReader

retriever = BM25Retriever(doc_store)  # BM25
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2',
                    context_window_size=1500,
                    use_gpu=True)

In [14]:
query='Who did Emma Marry?'
retriever.retrieve(query)

[<Document: {'content': "The Normans were in contact with England from an early date. Not only were their original Viking brethren still ravaging the English coasts, they occupied most of the important ports opposite England across the English Channel. This relationship eventually produced closer ties of blood through the marriage of Emma, sister of Duke Richard II of Normandy, and King Ethelred II of England. Because of this, Ethelred fled to Normandy in 1013, when he was forced from his kingdom by Sweyn Forkbeard. His stay in Normandy (until 1016) influenced him and his sons by Emma, who stayed in Normandy after Cnut the Great's conquest of the isle.", 'content_type': 'text', 'score': 0.8210750391274714, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '9b09ebf4eb65bd2133a6374e1f5ef5a'}>,
 <Document: {'content': "Some elements of the Brotherhood, though perhaps against orders, did engage in violence against the government, and its founder Al-Banna was assassinated in

And now we initialize our ODQA pipeline.

In [15]:
from haystack.pipelines import ExtractiveQAPipeline

qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

Now we can begin asking questions!

In [16]:
qa.run(query='Who did Emma Marry?')

Inferencing Samples: 100%|██████████| 1/1 [00:04<00:00,  4.19s/ Batches]


{'query': 'Who did Emma Marry?',
 'no_ans_gap': 8.314443588256836,
 'answers': [<Answer {'answer': 'King Ethelred II of England', 'type': 'extractive', 'score': 0.9178339242935181, 'context': "The Normans were in contact with England from an early date. Not only were their original Viking brethren still ravaging the English coasts, they occupied most of the important ports opposite England across the English Channel. This relationship eventually produced closer ties of blood through the marriage of Emma, sister of Duke Richard II of Normandy, and King Ethelred II of England. Because of this, Ethelred fled to Normandy in 1013, when he was forced from his kingdom by Sweyn Forkbeard. His stay in Normandy (until 1016) influenced him and his sons by Emma, who stayed in Normandy after Cnut the Great's conquest of the isle.", 'offsets_in_document': [{'start': 360, 'end': 387}], 'offsets_in_context': [{'start': 360, 'end': 387}], 'document_ids': ['9b09ebf4eb65bd2133a6374e1f5ef5a'], 'meta': {}}

In [12]:
qa.run(query='Who betrayed the Normans?',
       params={"Retriever": {"top_k": 5}, "Reader": {"top_k": 5}})

Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.33s/ Batches]


{'query': 'Who betrayed the Normans?',
 'no_ans_gap': 5.629459381103516,
 'answers': [<Answer {'answer': 'Venetian and Amalfitan merchants', 'type': 'extractive', 'score': 0.8696871995925903, 'context': "l, Robert continued his campaign conquering the Balkan peninsula as a foothold for western feudal lords and the Catholic Church. After allying himself with Croatia and the Catholic cities of Dalmatia, in 1081 he led an army of 30,000 men in 300 ships landing on the southern shores of Albania, capturing Valona, Kanina, Jericho (Orikumi), and reaching Butrint after numerous pillages. They joined the fleet that had previously conquered Corfu and attacked Dyrrachium from land and sea, devastating everything along the way. Under these harsh circumstances, the locals accepted the call of emperor Alexius I Comnenus to join forces with the Byzantines against the Normans. The Albanian forces could not take part in the ensuing battle because it had started before their arrival. Immediately befor

In [13]:


qa.run(query='Where was Dyrrachium located?',
       params={"Reader": {"top_k": 3}})

Inferencing Samples: 100%|██████████| 1/1 [00:03<00:00,  3.48s/ Batches]


{'query': 'Where was Dyrrachium located?',
 'no_ans_gap': 5.8214884996414185,
 'answers': [<Answer {'answer': 'Valona', 'type': 'extractive', 'score': 0.8835912942886353, 'context': "A few years after the First Crusade, in 1107, the Normans under the command of Bohemond, Robert's son, landed in Valona and besieged Dyrrachium using the most sophisticated military equipment of the time, but to no avail. Meanwhile, they occupied Petrela, the citadel of Mili at the banks of the river Deabolis, Gllavenica (Ballsh), Kanina and Jericho. This time, the Albanians sided with the Normans, dissatisfied by the heavy taxes the Byzantines had imposed upon them. With their help, the Normans secured the Arbanon passes and opened their way to Dibra. The lack of supplies, disease and Byzantine resistance forced Bohemond to retreat from his campaign and sign a peace treaty with the Byzantines in the city of Deabolis.", 'offsets_in_document': [{'start': 113, 'end': 119}], 'offsets_in_context': [{'start': 1